### Import modules

In [3]:
from datetime import date, timedelta
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import numpy as np
import torch

from JSONReader import read_json
from cov_matrix import MPTCovMat
from allocator import WeightOptimizer
from backtesting import repeatingBacktester

### Set parametes

In [4]:
# ---------- Testing ----------
eval_start = date(2000,1,1)
eval_end = date(2024,12,31)

lookback_period = 2 #years
rebalancing_interval = 0.5 #years

#loss function:
alpha = 0.3 #calmar
beta = 0.1 #omega
gamma = 0.6 #sortino

#target:
rf = 0.027 #risk free rate (can be used as target rate instead)
rf_period = timedelta(120) #number of days for risk free, e.g. 365 for annual 

num_iter = 10000 #number of weight update iterations
weight_decay = 0.1 #affects distribution of weights - higher value will lead to more even distribution

# ---------- COMPARISON ---------- 
comparison_ticker = '^OMX' #ticker of index to compare against

In [ ]:

#get data
data = MPTCovMat(tickers=tickers,start=eval_start,end=eval_end)
returns = data.log_return_df
col_indices = data.col_indices
asset_names = [col_indices[i] for i in sorted(col_indices.keys())]

allocator = WeightOptimizer(lr=1e-3,num_assets=len(asset_names),risk_free=rf,risk_free_period=rf_period,num_iter=num_iter, weight_decay=weight_decay)
cos_losses, pf_daily_returns = allocator.optimize_weights(alpha,beta,gamma, returns=torch.tensor(returns.values))
weights = pd.DataFrame(allocator.alloc_weights.numpy(force=True), index=asset_names, columns=['Weights'])
weights.index.name = 'Ticker'
weights['Weights %'] = weights['Weights']*100
weights['Weights %'].to_csv('weights.csv')
print(weights['Weights %'])

[*********************100%***********************]  30 of 30 completed


Optimizing weights... 
Ticker
ABB.ST          4.493760
ALFA.ST         2.003790
ASSA-B.ST       0.508921
ATCO-A.ST       1.664958
ATCO-B.ST       0.983837
AZN.ST          1.901111
BOL.ST          0.880206
ELUX-B.ST       0.207179
ERIC-B.ST       0.736760
ESSITY-B.ST     0.966924
EVO.ST          3.622478
GETI-B.ST       0.705915
HEXA-B.ST       0.330110
HM-B.ST         0.484249
INVE-B.ST      64.602547
KINV-B.ST       0.096646
NDA-SE.ST       0.181819
NIBE-B.ST       1.001772
SAAB-B.ST       1.031978
SAND.ST         0.287704
SBB-B.ST        0.951221
SCA-B.ST        1.867854
SEB-A.ST        1.718071
SHB-A.ST        0.441504
SINCH.ST        0.770154
SKF-B.ST        0.321081
SWED-A.ST       1.291334
TEL2-B.ST       3.096545
TELIA.ST        0.317192
VOLV-B.ST       2.532376
Name: Weights %, dtype: float32


In [ ]:
#create_dates
ticker_list = read_json('eval_assets.json')
tickers = yf.Tickers(ticker_list)

rebal_init = eval_start + timedelta(365*lookback_period)
current_date = rebal_init
while current_date <= eval_end:
    data = MPTCovMat(tickers=tickers,start=current_date,end=(current_date+1))
    allocator = WeightOptimizer(lr=1e-3,num_assets=len(asset_names),risk_free=rf,risk_free_period=rf_period,num_iter=num_iter, weight_decay=weight_decay)
    weights = pd.DataFrame(allocator.alloc_weights.numpy(force=True), index=asset_names, columns=['Weights'])
